In [1]:
# Importamos librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn import model_selection
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
import numpy as np 
import pandas as pd
from xgboost import XGBRegressor
import optuna
from sklearn.metrics import log_loss, mean_squared_error
from sklearn.metrics import accuracy_score
from scipy import stats


import warnings
warnings.filterwarnings("ignore")

In [2]:
test= pd.read_csv("../input/tabular-playground-series-feb-2022/test.csv")           
train_fold= pd.read_csv("../input/tps-feb-2022-kfold/train_folds.csv") 
sample_sub=pd.read_csv("../input/tabular-playground-series-feb-2022/sample_submission.csv")
data = pd.read_csv("../input/tabular-playground-series-feb-2022/train.csv")

In [3]:
targets = train_fold.target.unique()
target_dict = {t: i for i, t in enumerate(targets)} 
target_dict_inv = {v: k for k, v in target_dict.items()}
data["target_map"] = data.target.map(target_dict)

In [4]:
train_split, val_split = train_test_split(data,test_size=0.25,random_state=42)
train_split.shape,val_split.shape

((150000, 289), (50000, 289))

In [5]:
train_split.head()

,row_id,A0T0G0C10,A0T0G1C9,A0T0G2C8,A0T0G3C7,A0T0G4C6,A0T0G5C5,A0T0G6C4,A0T0G7C3,A0T0G8C2,...,A8T0G2C0,A8T1G0C1,A8T1G1C0,A8T2G0C0,A9T0G0C1,A9T0G1C0,A9T1G0C0,A10T0G0C0,target,target_map
21743,21743,-9.536743e-07,0.00001,0.000017,-0.000004,-0.00002,0.00008,-0.00003,0.000036,-0.000043,...,0.000157,0.000164,0.000194,0.000037,0.00001,0.00007,4.632568e-07,-9.536743e-07,Enterococcus_hirae,2
124554,124554,-9.536743e-07,-0.00001,-0.000043,-0.000114,-0.00020,-0.00024,-0.00020,-0.000114,-0.000043,...,-0.000043,-0.000086,-0.000086,-0.000043,-0.00001,-0.00001,-9.536743e-06,-9.536743e-07,Escherichia_fergusonii,7
10351,10351,-9.536743e-07,-0.00001,-0.000043,-0.000114,-0.00020,-0.00024,-0.00020,-0.000114,-0.000043,...,-0.000043,-0.000086,-0.000086,-0.000043,-0.00001,-0.00001,9.990463e-03,-9.536743e-07,Staphylococcus_aureus,6
135164,135164,-9.536743e-07,-0.00001,-0.000043,-0.000114,-0.00020,-0.00024,-0.00020,-0.000114,-0.000043,...,0.000957,-0.000086,0.000914,0.000957,-0.00001,-0.00001,-9.536743e-06,-9.536743e-07,Bacteroides_fragilis,8
49969,49969,-9.536743e-07,-0.00001,0.000007,0.000136,0.00039,0.00045,0.00044,0.000146,0.000027,...,0.000147,0.000224,0.000164,0.000177,0.00002,0.00001,3.046326e-05,-9.536743e-07,Salmonella_enterica,1


In [6]:
val_split.head()

,row_id,A0T0G0C10,A0T0G1C9,A0T0G2C8,A0T0G3C7,A0T0G4C6,A0T0G5C5,A0T0G6C4,A0T0G7C3,A0T0G8C2,...,A8T0G2C0,A8T1G0C1,A8T1G1C0,A8T2G0C0,A9T0G0C1,A9T0G1C0,A9T1G0C0,A10T0G0C0,target,target_map
119737,119737,-9.536743e-07,4.632568e-07,-0.000033,0.000016,0.000070,-0.000030,-0.000010,-0.000034,-0.000003,...,0.000117,0.000254,0.000154,0.000177,0.000020,0.000030,0.000040,-9.536743e-07,Escherichia_fergusonii,7
72272,72272,-9.536743e-07,1.046326e-05,0.000007,-0.000004,-0.000030,0.000020,-0.000050,0.000016,-0.000043,...,0.000137,0.000264,0.000324,0.000267,0.000010,0.000020,0.000050,-9.536743e-07,Staphylococcus_aureus,6
158154,158154,-9.536743e-07,-6.536743e-06,-0.000023,-0.000069,-0.000112,-0.000146,-0.000113,-0.000066,-0.000021,...,0.000312,0.000505,0.000557,0.000322,0.000064,0.000099,0.000086,1.046326e-06,Streptococcus_pyogenes,0
65426,65426,4.632568e-08,-8.536743e-06,-0.000018,0.000043,0.000030,-0.000007,0.000002,0.000047,-0.000019,...,0.000099,0.000199,0.000229,0.000168,0.000023,0.000015,0.000046,-9.536743e-07,Escherichia_fergusonii,7
30074,30074,-9.536743e-07,-9.536743e-06,-0.000043,-0.000114,-0.000200,-0.000240,0.009800,0.009886,-0.000043,...,-0.000043,-0.000086,-0.000086,0.009957,-0.000010,-0.000010,-0.000010,-9.536743e-07,Klebsiella_pneumoniae,9


In [7]:
features = [c for c in train_fold.columns if c not in ("target", "kfold","target_map")]
target="target_map"
df_test=test[features]

X_train= train_split[features]
y_train= train_split[target]
X_valid= val_split[features]
y_valid= val_split[target]
X_test= test[features]

In [8]:
def objective(trial):
    
    param = {
        'max_depth': trial.suggest_int('max_depth', 2, 15),
        'subsample': trial.suggest_discrete_uniform('subsample', 0.6, 1.0, 0.05),
        'n_estimators': trial.suggest_int('n_estimators', 1000, 10000, 100),
        'eta': trial.suggest_discrete_uniform('eta', 0.01, 0.1, 0.01),
        'reg_alpha': trial.suggest_int('reg_alpha', 1, 50),
        'reg_lambda': trial.suggest_int('reg_lambda', 5, 100),
        'min_child_weight': trial.suggest_int('min_child_weight', 2, 20),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.1, 1.0),
    }
   
    model = XGBClassifier(random_state=42, 
                             tree_method='gpu_hist', 
                             gpu_id=0, 
                             predictor="gpu_predictor"
                             ,**param )  
    model.fit(X_train, y_train,eval_set=[(X_valid,y_valid)], early_stopping_rounds=150,verbose=False)
    preds = model.predict(X_valid)
    acc = accuracy_score(y_valid, preds)
    return acc

In [9]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=3)
#study.optimize(objective, n_trials=50)
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

[I 2022-02-05 00:38:11,303] A new study created in memory with name: no-name-cf785b52-33c0-47d4-bc1d-5182e9d57708


[00:38:15] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


[I 2022-02-05 00:54:21,439] Trial 0 finished with value: 0.99298 and parameters: {'max_depth': 11, 'subsample': 0.7, 'n_estimators': 6200, 'eta': 0.02, 'reg_alpha': 2, 'reg_lambda': 24, 'min_child_weight': 12, 'colsample_bytree': 0.6539443755148}. Best is trial 0 with value: 0.99298.


[00:54:23] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


[I 2022-02-05 00:58:05,268] Trial 1 finished with value: 0.97954 and parameters: {'max_depth': 10, 'subsample': 0.75, 'n_estimators': 3100, 'eta': 0.08, 'reg_alpha': 31, 'reg_lambda': 64, 'min_child_weight': 4, 'colsample_bytree': 0.9833450613013808}. Best is trial 0 with value: 0.99298.


[00:58:06] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


[I 2022-02-05 01:02:28,586] Trial 2 finished with value: 0.9255 and parameters: {'max_depth': 2, 'subsample': 0.6, 'n_estimators': 6300, 'eta': 0.02, 'reg_alpha': 16, 'reg_lambda': 67, 'min_child_weight': 3, 'colsample_bytree': 0.8326740482741087}. Best is trial 0 with value: 0.99298.


Number of finished trials: 3
Best trial: {'max_depth': 11, 'subsample': 0.7, 'n_estimators': 6200, 'eta': 0.02, 'reg_alpha': 2, 'reg_lambda': 24, 'min_child_weight': 12, 'colsample_bytree': 0.6539443755148}


In [10]:
# Visualizamos la importancia de los parámetros
optuna.visualization.plot_param_importances(study)

In [11]:
####

In [12]:
test= pd.read_csv("../input/tabular-playground-series-feb-2022/test.csv")           
train_fold= pd.read_csv("../input/tps-feb-2022-kfold/train_folds.csv") 
sample_sub=pd.read_csv("../input/tabular-playground-series-feb-2022/sample_submission.csv")

In [13]:
targets = train_fold.target.unique()
target_dict = {t: i for i, t in enumerate(targets)} 
target_dict_inv = {v: k for k, v in target_dict.items()}
train_fold["target_map"] = train_fold.target.map(target_dict)
features = [c for c in train_fold.columns if c not in ("target", "kfold","target_map")]
target="target_map"
df_test=test[features]

In [14]:
params = {'max_depth': 10, 'subsample': 0.7, 'n_estimators': 5100, 'eta': 0.09999999999999999, 'reg_alpha': 1, 'reg_lambda': 73, 'min_child_weight': 5, 'colsample_bytree': 0.8377366123284274}

In [15]:
cv_preds_test = []
scores = []

for fold in range(7):
    X_train = train_fold[train_fold.kfold != fold].reset_index(drop=True)
    X_valid = train_fold[train_fold.kfold == fold].reset_index(drop=True)
    X_test = df_test.copy()
    
    y_train=X_train[target]
    y_valid=X_valid[target]
    
    
    X_train=X_train[features]
    X_valid=X_valid[features]
    
    # Encoding e ingeniería de funciones
    ##
    ##
    #
    
    
    # Entrenamiento
    model = XGBClassifier(random_state=fold, tree_method='gpu_hist', gpu_id=0, predictor="gpu_predictor",**params)
    model.fit(X_train,y_train)
    preds_valid=model.predict(X_valid)
    test_preds = model.predict(X_test)
    cv_preds_test.append(test_preds)
    acc = accuracy_score(y_valid, preds_valid)
    print(fold, acc)
    scores.append(acc)
    
print(f"La media es {np.mean(scores)} y la desviación estándar es {np.std(scores)}")

[01:02:52] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
0 0.995730085398292
[01:15:31] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
1 0.99597508049839
[01:28:10] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
2 0.9958700825983481
[01:40:48] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'

In [16]:
sample_sub['target'] =stats.mode(np.column_stack(cv_preds_test), axis=1)[0]
sample_sub['target']=sample_sub['target'].map(target_dict_inv)
sample_sub.head(10)
sample_sub.to_csv('submission.csv', index=False)